# Experiment Notebook

This notebook documents **experimental training and evaluation work** conducted during the development of the **Elderly Monitoring AI System**.

⚠️ **Important**  
- This notebook is for **research and experimentation only**.  
- It is **NOT part of the runtime system**.  
- The production inference code is located in `/src/layers/fire_detection`.


# Fire, Smoke, and Person Detection (YOLOv8)

This experiment trains and evaluates a **YOLOv8-based object detection model** capable of detecting:
- Fire
- Smoke
- Person

The model produced here is used **only for inference** in the final system.


## Purpose

- Merge multiple public datasets into a unified detection dataset
- Train a multi-class YOLOv8 model
- Evaluate detection performance on train, validation, and test splits


## Environment

- Python 3.10
- Ultralytics YOLOv8
- OpenCV
- NumPy
- KaggleHub

This notebook was designed for execution in **Google Colab**.


In [1]:
!pip install -q ultralytics kagglehub
print("✅ Dependencies installed")

✅ Dependencies installed



[notice] A new release of pip is available: 25.3 -> 26.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
import os
import shutil
import random
import requests
from collections import Counter
from io import BytesIO
from zipfile import ZipFile
import kagglehub

print("✅ Libraries imported")

## Dataset Download

This experiment uses:
- A public **person detection dataset** (Kaggle)
- An **indoor fire & smoke dataset** (Zenodo)


In [ ]:
def download_datasets():
    if not os.path.exists('/content/person'):
        path = kagglehub.dataset_download("samuelayman/person")
        shutil.copytree(path, "/content/person", dirs_exist_ok=True)
        print("Person dataset downloaded")
    else:
        print("Person dataset exists")

    if not os.path.exists('/content/smoke_fire'):
        url = "https://zenodo.org/records/15826133/files/Indoor%20Fire%20Smoke.zip"
        r = requests.get(url)
        ZipFile(BytesIO(r.content)).extractall('/content/smoke_fire')
        print("Fire/Smoke dataset downloaded")
    else:
        print("Fire/Smoke dataset exists")

download_datasets()

## Dataset Inspection

Basic checks are performed to verify dataset structure and file counts.


In [ ]:
!echo "=== Person Dataset ==="
!find /content/person -type f | head -10

!echo "\n=== Fire/Smoke Dataset ==="
!find /content/smoke_fire -type f | head -10

## Dataset Merging and Class Mapping

The datasets are merged into a single YOLO-compatible structure.

Final class mapping:
- `0` → fire
- `1` → person
- `2` → smoke


In [ ]:
if os.path.exists('merged_dataset'):
    shutil.rmtree('merged_dataset')

for split in ['train', 'val', 'test']:
    os.makedirs(f'merged_dataset/{split}/images', exist_ok=True)
    os.makedirs(f'merged_dataset/{split}/labels', exist_ok=True)


In [ ]:
person_img_subdirs = ['Train', 'Test', 'Validate']
person_base_img = '/content/person/person/Images'
person_base_lbl = '/content/person/person/Labels'

person_img_files = []
person_label_paths = {}

for sub in person_img_subdirs:
    img_dir = os.path.join(person_base_img, sub)
    lbl_dir = os.path.join(person_base_lbl, sub)

    if os.path.exists(img_dir):
        for f in os.listdir(img_dir):
            if f.lower().endswith(('.jpg', '.png')):
                person_img_files.append(os.path.join(img_dir, f))

    if os.path.exists(lbl_dir):
        for f in os.listdir(lbl_dir):
            if f.endswith('.txt'):
                person_label_paths[os.path.splitext(f)[0]] = os.path.join(lbl_dir, f)

fs_img_dir = '/content/smoke_fire/Indoor Fire Smoke/train/images'
fs_lbl_dir = '/content/smoke_fire/Indoor Fire Smoke/train/labels'
fs_img_files = [os.path.join(fs_img_dir, f) for f in os.listdir(fs_img_dir) if f.lower().endswith(('.jpg', '.png'))]

all_images = person_img_files + fs_img_files
random.shuffle(all_images)

In [ ]:
n = len(all_images)
train_end = int(0.7 * n)
val_end = int(0.9 * n)

splits = {
    'train': all_images[:train_end],
    'val': all_images[train_end:val_end],
    'test': all_images[val_end:]
}

counts = Counter()
for split, paths in splits.items():
    for img_path in paths:
        img_name = os.path.basename(img_path)
        shutil.copy(img_path, f'merged_dataset/{split}/images/{img_name}')

        base = os.path.splitext(img_name)[0]
        lbl_out = f'merged_dataset/{split}/labels/{base}.txt'

        if 'person' in img_path:
            src_lbl = person_label_paths.get(base)
        else:
            src_lbl = os.path.join(fs_lbl_dir, base + '.txt')

        if src_lbl and os.path.exists(src_lbl):
            with open(src_lbl) as f:
                lines = f.readlines()
            with open(lbl_out, 'w') as f:
                for line in lines:
                    parts = line.split()
                    old_class = int(parts[0])
                    new_class = 1 if 'person' in img_path else (0 if old_class == 0 else 2)
                    f.write(f"{new_class} {' '.join(parts[1:])}\n")
                    counts[new_class] += 1
        else:
            open(lbl_out, 'w').close()

## Dataset Summary


In [ ]:
for c, name in zip([0,1,2], ['fire','person','smoke']):
    print(f"Class {c} ({name}): {counts[c]}")

In [ ]:
data_yaml = """
path: /content/merged_dataset
train: train/images
val: val/images
test: test/images

nc: 3
names: ['fire', 'person', 'smoke']
"""

with open('data.yaml', 'w') as f:
    f.write(data_yaml.strip())

print("data.yaml created")

## Model Training


In [ ]:
from ultralytics import YOLO

model = YOLO('yolov8s.pt')

results = model.train(
    data='data.yaml',
    epochs=100,
    imgsz=640,
    batch=16,
    augment=True,
    patience=20,
    device=0,
    optimizer='AdamW',
    lr0=0.001,
    cos_lr=True,
    mixup=0.1,
    copy_paste=0.1
)

print("Training complete")

## Evaluation


In [ ]:
model = YOLO('runs/detect/train/weights/best.pt')
val_results = model.val(data='data.yaml', split='val')
test_results = model.val(data='data.yaml', split='test', plots=True)

print(f"Validation mAP@50-95: {val_results.box.map:.3f}")
print(f"Test mAP@50-95: {test_results.box.map:.3f}")

## Observations & Notes

- Dataset merging was a critical step for multi-class consistency.
- Augmentation improved smoke detection robustness.
- Final model balances accuracy and real-time performance.
- The trained weights are used **only for inference** in the final system.
